# Typical 1st & 2nd Order IIR-Filters in Audio Signal Processing

*This jupyter notebook is part of a [collection of notebooks](../index.ipynb) on various topics of Digital Signal Processing. Please direct questions and suggestions to [Sascha.Spors@uni-rostock.de](mailto:Sascha.Spors@uni-rostock.de).*

## Preliminaries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.markers import MarkerStyle
from matplotlib.patches import Circle
import scipy.signal as sig
np.set_printoptions(precision=16)
np.seterr(divide='ignore');  # log10 in bode plot generates divide by zero warning, which we will ignore

In [ ]:
fs = 48000  # set sampling frequency in Hz for upcoming simulations

Let's define some LaTex definitions for easier equation coding:

\def\im{\mathrm{i}} %imaginary unit, constant number, thus not italic
$\def\im{\mathrm{i}}$

\def\e{\mathrm{e}} %Euler's number, constant number, thus not italic
$\def\e{\mathrm{e}}$

We use a bibtex entry list at the end of the notebook and standard LaTex-style citing within the text. At the moment there is no long-term solution for rendering a reference list automatically, so we leave the information here as plain text. However, digital sources are linked with URLs.

## Introduction

We recollect the most common analog transfer functions (Laplace domain) of 1st and 2nd order IIR audio filters and derive bilinear transforms (transfer functions in z-domain), applying cut/mid frequency and quality/bandwidth prewarping.

Although more sophisticated filter designs exist, the discussed filter characteristics solve many IIR based audio filtering tasks, such as in digital audio workstation (DAW) and available [plugins](https://plugins.iem.at/), in digital mixing consoles, loudspeaker management systems, power amplifiers with DSP capabilities, audio player software.

Higher order IIR filters can be built from second order filters. A series connection of the discussed filter types achieves more complex transfer functions often being used in the mentioned applications.

### Standardized vs. Non-Standardized Filter Characteristics
For the **standardized** lowpass, highpass, bandpass, bandstop and allpass filters, the Laplace transfer functions and their coefficients are well known from literature e.g. \cite{Moschytz1981}, [\cite{TietzeSchenk2002}](http://www.tietze-schenk.de/).

For shelving and parametric filters **no standardization w.r.t. the pole/zero qualities and cut-frequencies and thus positions of poles and zeros** exists. Based on \cite{Zoelzer2002,Zoelzer2005},[\cite{Zoelzer2008}](https://link.springer.com/chapter/10.1007/978-3-540-34301-1_15),[\cite{Zoelzer2011}](https://doi.org/10.1002/9781119991298) (in which these filters are derived with the help of allpass filters) and some other articles \cite{Kimball1938}, \cite{Bristow-Johnson}, [\cite{Bristow-Johnson1994}](http://www.aes.org/e-lib/browse.cfm?elib=6326), [\cite{Orfanidis1997}](http://www.aes.org/e-lib/browse.cfm?elib=7854), [\cite{Christensen2003}](http://www.aes.org/e-lib/browse.cfm?elib=12429) widely used approaches can be found.

\cite{Christensen2003} discusses the generalized form of the biquad transfer function for progammable filters.

Three characteristics (labeled as types **I, II and III**) are most prominent, leading to different transfer functions for the same user parametrization. This regularly leads to misconceptions.

### Overview of Filter Transfer Functions

The follwing filter characteristics are discussed in this notebook:

[1st order lowpass](#lowpass1st) [2nd order lowpass](#lowpass2nd)

[1st order highpass](#highpass1st) [2nd order highpass](#highpass2nd)

[2nd order bandpass](#bandpass2nd) [2nd order bandstop](#bandstop2nd)

[1st order allpass](#allpass1st) [2nd order allpass](#allpass2nd)

[2nd order PEQ](#peq2nd) type **I,I,III**

[1st order low-shelving](#lowshv1st)

[2nd order low-shelving](#lowshv2nd) type **I,I,III**

[1st order high-shelving](#highshv1st)

[2nd order low-shelving](#highshv2nd) type **I,I,III**

## Bilinear Transformation for a IIR Biquad

The transformation of the analog filter prototypes (given as Laplace transfer function) towards the digital prototype (given as z-domain transfer function) is performed with the widely used [bilinear transform](bilinear_transform.ipynb) \cite[Ch. 7.1.2]{OppenheimSchaferBuck2004}) applying frequency and bandwidth prewarping techniques.

There are other approaches such as the impulse invariant method, the matched z-transform method and higher order Taylor series truncations \cite{Clark2000}, \cite{Gunness2007}, \cite{Alaoui2007}, \cite{Schmidt2010}, \cite{Alaoui2010} that yield higher precision matching of the analog and digital transform. For didactical purpose we restrict ourselves to the bilinear transform of IIR biquads.

Let's define the angular frequency $\omega$ in rad/s, $\im^2=-1$ and the sampling frequency $f_s$ in Hz. The connection between the Laplace domain and the z-domain is given by

\begin{equation}
s = \im\,\omega \qquad z = \e^{\frac{s}{f_s}} \qquad \stackrel{\text{linearization from Taylor series}}{\rightarrow} \qquad s = 2\,f_s\,\frac{z-1}{z+1}.
\end{equation}

The continuous and discrete time transfer functions $H(s)$ and $H(z)$, resp. for the 2nd order filter (biquad) are given as 

\begin{equation}
H(s) = \frac{B_0\,s^2+B_1\,s+B_2}{A_0\,s^2+A_1\,s+A_2}\\
\end{equation}

\begin{equation}
H(z) = \frac{b_0+b_1\,z^{-1}+b_2\,z^{-2}}{1+a_1\,z^{-1}+a_2\,z^{-2}}
\end{equation}

following typical conventions for the denotation of the coefficients (Python and Matlab compatible).

The bilinear transform rule $s = 2\,f_s\,\frac{z-1}{z+1}$ can be inserted to $H(s)$ and the result can be brought to form of biquad given as $H(z)$ above. Then the coefficients for the biquad are generally given by, cf. [bilinear transform](bilinear_transform.ipynb)

\begin{align}
b_0 &= \frac{B_2+2\,B_1\,f_s+4\,B_0\,f_s^2}{A_2+2\,A_1\,f_s+4\,A_0\,f_s^2}\newline
b_1 &= \frac{2\,B_2-8\,B_0\,f_s^2}{A_2+2\,A_1\,f_s+4\,A_0\,f_s^2}\newline
b_2 &= \frac{B_2-2\,B_1\,f_s+4\,B_0\,f_s^2}{A_2+2\,A_1\,f_s+4\,A_0\,f_s^2}\newline
a_1 &= \frac{2\,A_2-8\,A_0\,f_s^2}{A_2+2\,A_1\,f_s+4\,A_0\,f_s^2}\newline
a_2 &= \frac{A_2-2\,A_1\,f_s+4\,A_0\,f_s^2}{A_2+2\,A_1\,f_s+4\,A_0\,f_s^2}.
\end{align}

The function below realizes the bilinear transformation based on the above given equations.

In [ ]:
def get_bilinear_Biquad(B,A,fs):
    # bilinear transform of an IIR biquad
    # input coefficients:
    # 2nd order continuous-time filter:
    # B[0]=B0   B[1]=B1   B[2]=B2 
    # A[0]=A0   A[1]=A1   A[2]=A2
    #
    #        Y(s)   B0*s^2+B1*s+B2   B[0]*s^2+B[1]*s+B[2]
    # H(s) = ---- = -------------- = --------------------
    #        X(s)   A0*s^2+A1*s+A2   A[0]*s^2+A[1]*s+A[2]
    #
    # using bilinear transform H(s)->H(z) with s=2*fs*(z-1)/(z+1)
    #
    # output coefficients:
    # 2nd order discrete-time filter:
    # b[0]=b0   b[1]=b1   b[2]=b2 
    # a[0]=1    a[1]=a1   a[2]=a2
    #
    #        Y(z)   b2*z^-2+b1*z^-1+b0   b[2]*z^-2+b[1]*z^-1+b[0]
    # H(z) = ---- = ------------------ = ------------------------
    #        X(z)   a2*z^-2+a1*z^-1+ 1   a[2]*z^-2+a[1]*z^-1+a[0]    
   
    a = np.array([0.,0.,0.])  # alloc RAM
    b = np.array([0.,0.,0.])
    fs2 = fs**2; #helper
    
    tmp  = A[2] + 2*A[1]*fs + 4*A[0]*fs2;
    b[0] = B[2] + 2*B[1]*fs + 4*B[0]*fs2;

    b[1] = 2*B[2] - 8*B[0]*fs2;
    a[1] = 2*A[2] - 8*A[0]*fs2;

    b[2] = B[2] - 2*B[1]*fs + 4*B[0]*fs2;
    a[2] = A[2] - 2*A[1]*fs + 4*A[0]*fs2;

    a = a / tmp #normalize for a[0]=1
    b = b / tmp
    a[0] = 1. #set
    
    return b,a

## Frequency and Bandwidth Prewarping

A so called prewarping \cite[eq. (7.28)]{OppenheimSchaferBuck2004} of the cut/mid frequency $f_c$/$f_m$ in Hz

\begin{equation}
\omega_{c} \Leftarrow 2\,f_s\,\tan{\left(\pi\frac{f_{c}}{f_s}\right)}
\end{equation}

\begin{equation}
\omega_{m} \Leftarrow 2\,f_s\,\tan{\left(\pi\frac{f_{m}}{f_s}\right)}
\end{equation}

is realized in the function below.

In [ ]:
def get_Frequency_prewarping(f,fs):
    wp = 2*fs*np.tan(np.pi*f/fs)  # pre-warped angular frequency for cut/mid frequency fc/fm
    return wp  # in rad/s

The prewarping of the so-called quality for the parametric, bandpass and bandstop filters can be done with

* **tan**-prewarping, cf. [\cite{Clark2000}](http://www.aes.org/e-lib/browse.cfm?elib=12069)

\begin{equation}
Q_\text{BP} \Leftarrow \frac{\frac{\pi\,f_m}{f_s}}{\tan\left(\frac{\pi\,f_m}{f_s}\right)}\cdot Q_\text{BP}
\end{equation}

* **cos**-prewarping, cf. \cite{Thaden1997}

\begin{equation}
Q_\text{BP} \Leftarrow \cos(\pi\,\frac{f_m}{f_s})\cdot Q_\text{BP}
\end{equation}

* **sin**-prewarping of the bandwidth in octaves, cf. \cite{Bristow-Johnson1994}

\begin{equation}
BW_\text{oct} \Leftarrow \frac{2\,\pi\,\frac{f_m}{f_s}}{\sin(2\,\pi\,\frac{f_m}{f_s})}\cdot BW_\text{oct}
\end{equation}

exhibiting slightly different characteristics.


In [ ]:
def get_Q_prewarping(Q,fm,fs,WarpType):
    if WarpType == "sin":
        # Robert Bristow-Johnson (1994): "The equivalence of various methods of 
        # computing biquad coefficients for audio parametric equalizers." 
        # In: Proc. of 97th AES Convention, San Fransisco
        # eq. (14)
        w0 = 2*np.pi*fm/fs
        BW = get_BW_from_Q(Q)
        BW = BW * w0/np.sin(w0)
        Qp = get_Q_from_BW(BW)        
    elif WarpType == "cos":
        # Rainer Thaden (1997): "Entwicklung und Erprobung einer digitalen 
        # parametrischen Filterbank." Diplomarbeit, RWTH Aachen
        Qp = Q * np.cos(np.pi*fm/fs)
    elif WarpType == "tan":
        # Clark, R.J.; Ifeachor, E.C.; Rogers, G.M.; et al. (2000): Techniques for
        # Generating Digital Equalizer Coefficients. In J. Aud. Eng. Soc. 48(4):281-298.
        Qp = Q * (np.pi*fm/fs) / (np.tan(np.pi*fm/fs))
    else:
        # no prewarping
        Qp = Q
        
    return Qp

The connection between bandwidth $BW_\text{oct}$ in octaves and corresponding bandpass-related quality $Q_\text{BP}$ is given as

\begin{equation}
BW_\text{oct} = \frac{2}{\log_\e(2)}\,\sinh^{-1}(\frac{1}{2\,Q_\text{BP}})
\end{equation}

\begin{equation}
Q_\text{BP} = \frac{1}{2\,\sinh(\frac{\log_\e(2)}{2}\,BW_\text{oct})}
\end{equation}

with natural logarithm $\log_\e(\cdot)$ and hyperbolic sine $\sinh(\cdot)$.

In [ ]:
def get_BW_from_Q(Q):
    BW = 2/np.log(2) * np.arcsinh(1/(2*Q))
    return BW
    
def get_Q_from_BW(BW):
    Q = 1 / (2*np.sinh(np.log(2)/2*BW))
    return Q

## Plotting Routine
We establish a plotting routine for the bode plot, i.e. the magnitude and phase spectrum of the filter's transfer function. For that we use the freqs() and freqz() functions implemented in the signal package. We also use the self-implemented zplane() function for plotting zeros and poles in the z-plane.

In [ ]:
def zplane(z, p, ax, title='Poles x and zeros o of discrete time domain'):
    "Plots zero and pole locations in the complex z-plane"

    ax.plot(np.real(z), np.imag(z), 'bo', fillstyle='none', ms = 10)
    ax.plot(np.real(p), np.imag(p), 'rx', fillstyle='none', ms = 10)
    unit_circle = Circle((0,0), radius=1, fill=False,
                         color='black', ls='solid', alpha=0.9)
    ax.add_patch(unit_circle)
    ax.axvline(0, color='0.7')
    ax.axhline(0, color='0.7')
    
    plt.title(title)
    plt.xlabel(r'Re{$z$}')
    plt.ylabel(r'Im{$z$}')
    plt.xlim((-1, +1))
    plt.ylim((-1, +1))
    plt.xticks(np.arange(-1, 1.25, step=0.25))
    plt.yticks(np.arange(-1, 1.25, step=0.25))    
    plt.axis('square')
    plt.grid()

In [ ]:
def get_Bode_plot(B,A,b,a,fs):
    W, Hd = sig.freqz(b, a, 2**12)
    s, Ha = sig.freqs(B, A, fs*W)
    f = fs*W / (2*np.pi)
    
    p = np.roots(a)
    z = np.roots(b)
    
    plt.figure(figsize = (16,9))
    
    plt.subplot(2,2,1)
    plt.semilogx(f, 20*np.log10(np.abs(Ha)),'b', label=r'$|H(\omega)|$ continuous-time', linewidth=3)
    plt.semilogx(f, 20*np.log10(np.abs(Hd)),'r', label=r'$|H(\Omega)|$ discrete-time')
    plt.title(r'fs=%5.f Hz' %fs)
    #plt.xlabel(r'f / Hz')
    plt.ylabel(r'20 log10 |H| / dB')
    plt.axis([10, 20000, -15, +15])
    plt.yticks(np.arange(-15,+15+3,3));
    plt.grid();
    plt.legend(loc=3)
   
    plt.subplot(2,2,3)
    plt.semilogx(f, (np.angle(Ha)*180/np.pi),'b', linewidth=3)
    plt.semilogx(f, (np.angle(Hd)*180/np.pi),'r')
    #plt.title(r'fs=%5.f Hz' %fs)
    plt.xlabel(r'f / Hz')
    plt.ylabel(r'angle(H) / rad')
    plt.axis([10, 20000, -np.pi, +np.pi])
    plt.yticks(np.arange(-180,+180+45,45));
    plt.grid();
    
    ax = plt.subplot(2,2,(2,4))
    zplane(z,p,ax)
   

## Example

### Without Prewarping
A first example shows the application of the bilinear transform and plotting routine. We use a lowpass filter (basic RC circuit) with the cut frequency $f_c$ in Hz, which is linked to $\omega_c=2\,\pi\,f_c=\frac{1}{R\,C}$. For the discrete time filter we need to specify the sampling frequency $f_s$ in Hz.

In [ ]:
# example lowpass 1st order:
fc = 10000  # cut frequency in Hz
wg = 2*np.pi*fc  # cut frequency in rad/s

B = np.array([0.,0.,1.])  # Laplace transfer function coefficients B
A = np.array([0.,1./wg,1.])  # Laplace transfer function coefficients A

[b,a] = get_bilinear_Biquad(B,A,fs)  # get z-transfer function coefficients based on bilinear transform
[b1,a1] = sig.bilinear(B,A,fs)  # use scipy.signal function

In [ ]:
b-b1;  # check if functions sig.bilinear and get_bilinear_Biquad doing the same for a biquad

In [ ]:
a-a1;  # check if functions sig.bilinear and get_bilinear_Biquad doing the same for a biquad 

In [ ]:
get_Bode_plot(B,A,b,a,fs)  # get bode plot

### With Prewarping

Now, do the same with frequency prewarping, in order that the intended 'analog' $f_c$ matches the 'digital' $f_c$ in discrete time domain:

In [ ]:
# example lowpass 1st order with frequency pre-warping:
fc = 10000
wc =2*np.pi*fc
B = np.array([0.,0.,1.])  # continuous time coefficients as above
A = np.array([0.,1./wc,1.])
wp = get_Frequency_prewarping(fc,fs)  # apply frequency prewarping here
Bp = np.array([0.,0.,1.])  # discrete-time coefficients after prewarping
Ap = np.array([0.,1./wp,1.])
[b,a] = get_bilinear_Biquad(Bp,Ap,fs) 
get_Bode_plot(B,A,b,a,fs)

## Definition of 1st/2nd Order Filter Transfer Functions

Below we implement functions for lowpass, highpass, allpass, bandpass, bandstop, shelving and parametric filters. These characteristics are very often used for audio, cf. e.g. [\cite{TietzeSchenk2002}](http://www.tietze-schenk.de/), \cite{Moschytz1981}, \cite{Christensen2003}, \cite{Zoelzer2008}, \cite{Zoelzer2011}, \cite{Bristow-Johnson1994}

In [ ]:
def get_coeff_LP1st(fc,fs):  # lowpass 1st order
    # cut frequency fc in Hz
    # sampling frequency fs in Hz
    wc = 2*np.pi*fc
    B = np.array([0., 0., 1.])
    A = np.array([0., 1. / wc, 1.])
    
    wp = get_Frequency_prewarping(fc,fs)
    Bp = np.array([0., 0., 1.])
    Ap = np.array([0., 1. / wp, 1.])
    [b,a] = get_bilinear_Biquad(Bp,Ap,fs)
 
    return B,A,b,a



def get_coeff_LP2nd(fc,fs,bi,ai):  # lowpass 2nd order
    # cut frequency fc in Hz
    # sampling frequency fs in Hz
    # bi,ai filter characteristics coefficients
    # such as, e.q.
    # bi = 0.6180 ai = 1.3617 for Bessel
    # bi = 1.     ai = 1.4142 for Butterworth
    wc = 2*np.pi*fc
    B = np.array([0., 0., 1.])
    A = np.array([bi / (wc**2), ai / wc, 1.])
    
    wp = get_Frequency_prewarping(fc,fs)
    Bp = np.array([0., 0., 1.])
    Ap = np.array([bi / (wp**2), ai / wp, 1.])
    [b,a] = get_bilinear_Biquad(Bp,Ap,fs)
    
    return B,A,b,a



def get_coeff_HP1st(fc,fs):  # highpass 1st order
    # cut frequency fc in Hz
    # sampling frequency fs in Hz    
    wc = 2*np.pi*fc
    B = np.array([0., 1. / wc, 0.])
    A = np.array([0., 1. / wc, 1.])
    
    wp = get_Frequency_prewarping(fc,fs)
    Bp = np.array([0., 1. / wp, 0.])
    Ap = np.array([0., 1. / wp, 1.])
    
    [b,a] = get_bilinear_Biquad(Bp,Ap,fs)
    return B,A,b,a



def get_coeff_HP2nd(fc,fs,bi,ai):  # highpass 2nd order
    # cut frequency fc in Hz
    # sampling frequency fs in Hz
    # bi,ai filter characteristics coefficients
    # such as e.q.
    # bi = 0.6180 ai = 1.3617 for Bessel
    # bi = 1.     ai = 1.4142 for Butterworth  
    wc = 2*np.pi*fc
    B = np.array([1. / (wc**2), 0., 0.])
    A = np.array([1. / (wc**2), ai / wc, bi])
    
    wp = get_Frequency_prewarping(fc,fs)
    Bp = np.array([1. / (wp**2), 0., 0.])
    Ap = np.array([1. / (wp**2), ai / wp, bi])
    
    [b,a] = get_bilinear_Biquad(Bp,Ap,fs)
    return B,A,b,a



def get_coeff_BP2nd(fm,Q,fs,QWarpType):  # bandpass 2nd order
    # mid frequency fm in Hz
    # quality Q
    # sampling frequency fs in Hz
    # QWarpType "sin", "cos" or "tan"
    wm = 2*np.pi*fm
    B = np.array([0., 1. / (Q*wm), 0.])
    A = np.array([1. / (wm**2), 1. / (Q*wm), 1.])    
    
    wp = get_Frequency_prewarping(fm,fs)
    Qp = get_Q_prewarping(Q,fm,fs,QWarpType)
    Bp = np.array([0., 1. / (Qp*wp), 0.])
    Ap = np.array([1. / (wp**2), 1. / (Qp*wp), 1.]) 
    [b,a] = get_bilinear_Biquad(Bp,Ap,fs)
    
    return B,A,b,a



def get_coeff_BS2nd(fm,Q,fs,QWarpType):  # bandstop 2nd order
    # mid frequency fm in Hz
    # quality Q
    # sampling frequency fs in Hz
    # QWarpType "sin", "cos" or "tan"    
    wm = 2*np.pi*fm
    B = np.array([1. / wm**2, 0., 1.])
    A = np.array([1. / wm**2, 1. / (Q*wm), 1.])    
    
    wp = get_Frequency_prewarping(fm,fs)
    Qp = get_Q_prewarping(Q,fm,fs,QWarpType)
    Bp = np.array([1. / wp**2, 0., 1.])
    Ap = np.array([1. / wp**2, 1. / (Qp*wp), 1.])  
    [b,a] = get_bilinear_Biquad(Bp,Ap,fs)
    
    return B,A,b,a    



def get_coeff_All1st(fc,ai,fs):  # allpass 1st order
    # cut frequency fc in Hz
    # bi,ai filter characteristics coefficients
    # such as e.q.
    # ai = 1.
    # sampling frequency fs in Hz 
    wc = 2*np.pi*fc
    B = np.array([0., -ai / wc, 1.])
    A = np.array([0., +ai / wc, 1.])
    
    wp = get_Frequency_prewarping(fc,fs)
    Bp = np.array([0., -ai / wp, 1.])
    Ap = np.array([0., +ai / wp, 1.])
    [b,a] = get_bilinear_Biquad(Bp,Ap,fs)

    return B,A,b,a



def get_coeff_All2nd(fc,bi,ai,fs):  # allpass 2nd order
    # cut frequency fc in Hz
    # bi,ai filter characteristics coefficients
    # such as e.q.
    # bi = 1 ai = 1.4142 
    # sampling frequency fs in Hz 
    wc = 2*np.pi*fc
    B = np.array([bi / (wc**2), -ai / wc, 1.])
    A = np.array([bi / (wc**2), +ai / wc, 1.])
    
    wp = get_Frequency_prewarping(fc,fs)
    Bp = np.array([bi/(wp**2.), -ai/wp, 1.])
    Ap = np.array([bi/(wp**2.), +ai/wp, 1.])   
    [b,a] = get_bilinear_Biquad(Bp,Ap,fs)

    return B,A,b,a



def get_coeff_PEQ2nd(fm,G,Q,fs,PEQType,QWarpType):  # parametric equalizer (PEQ) 2nd order
    # mid frequency fm in Hz
    # gain G in dB
    # quality Q
    # sampling frequency fs in Hz
    # PEQType "I", "II", "III"
    # QWarpType "sin", "cos" or "tan"
    wm = 2*np.pi*fm
    g = 10**(G/20)
    QBP = Q
    
    if PEQType == "I":  # aka constant-Q PEQ
        gamma = g
        delta = g
    elif PEQType == "II":  # aka symmetrical PEQ
        gamma = 1.
        delta = g
    elif PEQType == 'III':  # aka one-half pad loss PEQ
        gamma = g**0.5
        delta = g**0.5
    else:
        gamma = unknown_PEQType
        delta = unknown_PEQType
    
    if G > 0.:
        B = np.array([1. / wm**2, delta / (QBP*wm), 1.])
        A = np.array([1. / wm**2, (delta/g) / (QBP*wm), 1.])        
    else:
        B = np.array([1. / wm**2, gamma / (QBP*wm), 1.])
        A = np.array([1. / wm**2, (gamma/g) / (QBP*wm), 1.])
    
    wp = get_Frequency_prewarping(fm,fs)
    Qp = get_Q_prewarping(QBP,fm,fs,QWarpType)
    
    if G > 0.:
        Bp = np.array([1. / wp**2, delta / (Qp*wp), 1.])
        Ap = np.array([1. / wp**2, (delta/g) / (Qp*wp), 1.])        
    else:
        Bp = np.array([1. / wp**2, gamma / (Qp*wp), 1.])
        Ap = np.array([1. / wp**2, (gamma/g) / (Qp*wp), 1.])
    [b,a] = get_bilinear_Biquad(Bp,Ap,fs)
    
    if np.isclose(G, 0., rtol=1e-05, atol=1e-08, equal_nan=False):  # if G==0 dB
        B = np.array([0.,0.,1.])  # make flat
        A = np.array([0.,0.,1.])         
        b = np.array([1.,0.,0.])
        a = np.array([1.,0.,0.])     
    
    return B,A,b,a       



def get_coeff_LShv1st(fc,G,fs,ShvType):  # lowshelving filter 1st order
    # cut frequency fc in Hz
    # gain G in dB
    # sampling frequency fs in Hz
    # ShvType "I", "II", "III"
    g = 10**(G/20)
    if ShvType == "I":
        alpha = 1.
    elif ShvType == "II":
        alpha = g**0.5
    elif ShvType == "III":  # one-half pad loss characteristics
        alpha = g**0.25
    else:
        alpha = unknown_ShvType
    
    wc = 2*np.pi*fc
    if G > 0.:
        B = np.array([0., 1. / wc, g * alpha**-2])
        A = np.array([0., 1. / wc, alpha**-2]) 
    else:
        B = np.array([0., 1. / wc, alpha**2])
        A = np.array([0., 1. / wc, g**-1 * alpha**2]) 
        
    wp = get_Frequency_prewarping(fc,fs)    
    if G > 0.:
        Bp = np.array([0., 1. / wp, g * alpha**-2])
        Ap = np.array([0., 1. / wp, alpha**-2]) 
    else:
        Bp = np.array([0., 1. / wp, alpha**2])
        Ap = np.array([0., 1. / wp, g**-1 * alpha**2])         
    [b,a] = get_bilinear_Biquad(Bp,Ap,fs)

    if np.isclose(G, 0., rtol=1e-05, atol=1e-08, equal_nan=False):  # if G==0 dB
        B = np.array([0.,0.,1.])  # make flat
        A = np.array([0.,0.,1.])         
        b = np.array([1.,0.,0.])
        a = np.array([1.,0.,0.])  
    return B,A,b,a



def get_coeff_LShv2nd(fc,G,Qz,Qp,fs,ShvType):  # lowshelving filter 2nd order
    # cut frequency fc in Hz
    # gain G in dB
    # zero Quality Qz, use e.g. Qz = 1./np.sqrt(2.) # Butterworth quality
    # pole quality Qp, use e.g. Qp = 1./np.sqrt(2.) # Butterworth quality
    # sampling frequency fs in Hz
    # ShvType "I", "II", "III"
    g = 10**(G/20)
    if ShvType == "I":
        alpha = 1.
    elif ShvType == "II":
        alpha = g**0.5
    elif ShvType == "III":  # one-half pad loss characteristics
        alpha = g**0.25       
    else:
        alpha = unknown_ShvType

    wc = 2*np.pi*fc;
    if G > 0.:
        B = np.array([1. / wc**2, g**0.5 * alpha**-1 / (Qz*wc), g * alpha**-2])
        A = np.array([1. / wc**2, alpha**-1 / (Qp*wc), alpha**-2])    
    else:
        B = np.array([1. / wc**2, alpha / (Qz*wc), alpha**2])
        A = np.array([1. / wc**2, g**-0.5 * alpha / (Qp*wc), g**-1 * alpha**2])    
     
    wp = get_Frequency_prewarping(fc,fs)    
    if G > 0.:
        Bp = np.array([1. / wp**2, g**0.5 * alpha**-1 / (Qz*wp), g * alpha**-2])
        Ap = np.array([1. / wp**2, alpha**-1 / (Qp*wp), alpha**-2])    
    else:
        Bp = np.array([1. / wp**2, alpha / (Qz*wp), alpha**2])
        Ap = np.array([1. / wp**2, g**-0.5 * alpha / (Qp*wp), g**-1 * alpha**2])     
    [b,a] = get_bilinear_Biquad(Bp,Ap,fs)
    
    if np.isclose(G, 0., rtol=1e-05, atol=1e-08, equal_nan=False):  # if G==0 dB
        B = np.array([0.,0.,1.])  # make flat
        A = np.array([0.,0.,1.])         
        b = np.array([1.,0.,0.])
        a = np.array([1.,0.,0.])     
    
    return B,A,b,a 



def get_coeff_HShv1st(fc,G,fs,ShvType):  # highshelving filter 1st order
    # cut frequency fc in Hz
    # gain G in dB
    # sampling frequency fs in Hz
    # ShvType "I", "II", "III"
    g = 10**(G/20)
    if ShvType == "I":
        alpha = 1.
    elif ShvType == "II":
        alpha = g**0.5
    elif ShvType == "III":  # one-half pad loss characteristics
        alpha = g**0.25
    else:
        alpha = unknown_ShvType
    
    wc = 2*np.pi*fc
    if G > 0.:
        B = np.array([0., g * alpha**-2 / wc, 1.])
        A = np.array([0., alpha**-2 / wc, 1.]) 
    else:
        B = np.array([0., alpha**2 / wc, 1.])
        A = np.array([0., g**-1 * alpha**2 / wc, 1.]) 
        
    wp = get_Frequency_prewarping(fc,fs)    
    if G > 0.:
        Bp = np.array([0., g * alpha**-2 / wp, 1.])
        Ap = np.array([0., alpha**-2 / wp, 1.]) 
    else:
        Bp = np.array([0., alpha**2 / wp, 1.])
        Ap = np.array([0., g**-1 * alpha**2 / wp, 1.])     
    [b,a] = get_bilinear_Biquad(Bp,Ap,fs)

    if np.isclose(G, 0., rtol=1e-05, atol=1e-08, equal_nan=False):  # if G==0 dB
        B = np.array([0.,0.,1.])  # make flat
        A = np.array([0.,0.,1.])         
        b = np.array([1.,0.,0.])
        a = np.array([1.,0.,0.])  
    return B,A,b,a



def get_coeff_HShv2nd(fc,G,Qz,Qp,fs,ShvType):  # highshelving filter 2nd order
    # cut frequency fc in Hz
    # gain G in dB
    # zero Quality Qz, use e.g. Qz = 1./np.sqrt(2.) # Butterworth quality
    # pole quality Qp, use e.g. Qp = 1./np.sqrt(2.) # Butterworth quality
    # sampling frequency fs in Hz
    # ShvType "I", "II", "III"
    g = 10**(G/20)
    if ShvType == "I":
        alpha = 1.
    elif ShvType == "II":
        alpha = g**+0.5
    elif ShvType == "III":  # one-half pad loss characteristics
        alpha = g**0.25       
    else:
        alpha = unknown_ShvType

    wc = 2*np.pi*fc;
    if G > 0.:
        B = np.array([g * alpha**-2 / wc**2., g**0.5 * alpha**-1 / (Qz*wc), 1.])
        A = np.array([alpha**-2 / wc**2, alpha**-1 / (Qp*wc), 1.])    
    else:
        B = np.array([alpha**2 / wc**2, alpha / (Qz*wc), 1.])
        A = np.array([g**-1 * alpha**2 / wc**2., g**-0.5 * alpha / (Qp*wc), 1.])    
     
    wp = get_Frequency_prewarping(fc,fs)    
    if G > 0.:
        Bp = np.array([g * alpha**-2 / wp**2, g**0.5 * alpha**-1 / (Qz*wp), 1.])
        Ap = np.array([alpha**-2 / wp**2, alpha**-1 / (Qp*wp), 1.])    
    else:
        Bp = np.array([alpha**2 / wp**2, alpha / (Qz*wp), 1.])
        Ap = np.array([g**-1 * alpha**2 / wp**2, g**-0.5 * alpha/(Qp*wp), 1.])    
    [b,a] = get_bilinear_Biquad(Bp,Ap,fs)
    
    if np.isclose(G, 0., rtol=1e-05, atol=1e-08, equal_nan=False):  # if G==0 dB
        B = np.array([0.,0.,1.])  # make flat
        A = np.array([0.,0.,1.])         
        b = np.array([1.,0.,0.])
        a = np.array([1.,0.,0.])     
    
    return B,A,b,a              

<a id="lowpass1st"></a>
## 1st Order Lowpass

In [ ]:
fc = 1000  # cut frequency in Hz
[B,A,b,a] = get_coeff_LP1st(fc,fs)
get_Bode_plot(B,A,b,a,fs)
print("B=",B)
print("A=",A)
print("b=",b)
print("a=",a)

<a id="lowpass2nd"></a>
## 2nd Order Lowpass

In [ ]:
fc = 1000  # cut frequency in Hz
bi = 1  # filter characteristic coeff
ai = np.sqrt(2)  # filter characteristic coeff, i.e. Butterworth
[B,A,b,a] = get_coeff_LP2nd(fc,fs,bi,ai)
get_Bode_plot(B,A,b,a,fs)
print("B=",B)
print("A=",A)
print("b=",b)
print("a=",a)

<a id="highpass1st"></a>
## 1st Order Highpass

In [ ]:
fc = 100  # cut frequency in Hz
[B,A,b,a] = get_coeff_HP1st(fc,fs)
get_Bode_plot(B,A,b,a,fs)
print("B=",B)
print("A=",A)
print("b=",b)
print("a=",a)

<a id="highpass2nd"></a>
## 2nd Order Highpass

In [ ]:
fc = 100  # cut frequency in Hz
bi = 1  # filter characteristic coeff
ai = np.sqrt(2)  # filter characteristic coeff, i.e. Butterworth
[B,A,b,a] = get_coeff_HP2nd(fc,fs,bi,ai)
get_Bode_plot(B,A,b,a,fs)
print("B=",B)
print("A=",A)
print("b=",b)
print("a=",a)

<a id="bandpass2nd"></a>
## 2nd Order Bandpass

In [ ]:
BW = 2  # bandwidth in octaves
fm = 1000  # mid frequency in Hz

flow =  2**(-BW/2)*fm  # lower cut  (-3 dB) frequency in Hz 
fhigh = 2**(+BW/2)*fm  # higher cut (-3 dB) frequency in Hz 

QBP = fm/(fhigh-flow) #is identical with:
QBP = get_Q_from_BW(BW)

QWarpType = "cos" # sin, cos, tan
[B,A,b,a] = get_coeff_BP2nd(fm,QBP,fs,QWarpType)
get_Bode_plot(B,A,b,a,fs)


print("flow=",flow,"Hz")
print("fhigh=",fhigh,"Hz")
print("QBP=",QBP)
print("B=",B)
print("A=",A)
print("b=",b)
print("a=",a)

<a id="bandstop2nd"></a>
## 2nd Order Bandstop

In [ ]:
fm = 1000  # mid frequency in Hz
Q = 2/3  # quality
QWarpType = "cos" # sin, cos, tan
[B,A,b,a] = get_coeff_BS2nd(fm,Q,fs,QWarpType)
get_Bode_plot(B,A,b,a,fs)

print("flow=",flow,"Hz")
print("fhigh=",fhigh,"Hz")
print("QBP=",QBP)
print("B=",B)
print("A=",A)
print("b=",b)
print("a=",a)

<a id="allpass1st"></a>
## 1st Order Allpass

In [ ]:
fc = 1000  # cut frequency in Hz
ai =1  # quality coeff
[B,A,b,a] = get_coeff_All1st(fc,ai,fs)
get_Bode_plot(B,A,b,a,fs)

print("B=",B)
print("A=",A)
print("b=",b)
print("a=",a)

<a id="allpass2nd"></a>
## 2nd Order Allpass

In [ ]:
fc = 1000  # cut frequency in Hz
bi = 1  # filter characteristic coeff
ai = np.sqrt(2) #filter characteristic coeff, i.e. Butterworth like
[B,A,b,a] = get_coeff_All2nd(fc,bi,ai,fs)
get_Bode_plot(B,A,b,a,fs)

print("B=",B)
print("A=",A)
print("b=",b)
print("a=",a)

<a id="peq2nd"></a>
## 2nd Order PEQ

In [ ]:
BW = 2  # bandwidth in octaves
fm = 1000  # mid frequency in Hz
G = 12  # gain in dB
Q = get_Q_from_BW(BW)
PEQType = "III" # I,II,III
QWarpType = "cos" # sin, cos, tan
[B,A,b,a] = get_coeff_PEQ2nd(fm,G,Q,fs,PEQType,QWarpType)
get_Bode_plot(B,A,b,a,fs)

print("B=",B)
print("A=",A)
print("b=",b)
print("a=",a)

<a id="lowshv1st"></a>
## 1st Order Low-Shelving

In [ ]:
fc =1000  # cut frequency in Hz
G = 12  # gain in dB
ShvType = "III" # I,II,III
[B,A,b,a] = get_coeff_LShv1st(fc,G,fs,ShvType)
get_Bode_plot(B,A,b,a,fs)

print("B=",B)
print("A=",A)
print("b=",b)
print("a=",a)

<a id="lowshv2nd"></a>
## 2nd Order Low-Shelving

In [ ]:
fc = 1000  # cut frequency in Hz
G = 12  # gain in dB
ShvType = "III" # I,II,III
Qz = 1/np.sqrt(2)
Qp = Qz
[B,A,b,a] = get_coeff_LShv2nd(fc,G,Qz,Qp,fs,ShvType)
get_Bode_plot(B,A,b,a,fs)

print("B=",B)
print("A=",A)
print("b=",b)
print("a=",a)

<a id="highshv1st"></a>
## 1st Order High-Shelving

In [ ]:
fc =1000  # cut frequency in Hz
G = 12  # gain in dB
ShvType = "III" #I, II, III
[B,A,b,a] = get_coeff_HShv1st(fc,G,fs,ShvType)
get_Bode_plot(B,A,b,a,fs)

print("B=",B)
print("A=",A)
print("b=",b)
print("a=",a)

<a id="highshv2nd"></a>
## 2nd Order High-Shelving

In [ ]:
fc = 1000  # cut frequency in Hz
G = 12  # gain in dB
ShvType = "III" #I, II, III
Qz = 1/np.sqrt(2) # Butterworth quality
Qp = Qz #dito
[B,A,b,a] = get_coeff_HShv2nd(fc,G,Qz,Qp,fs,ShvType)
get_Bode_plot(B,A,b,a,fs)

print("B=",B)
print("A=",A)
print("b=",b)
print("a=",a)

In [ ]:
fc = 1000  # cut frequency in Hz
G = 12  # gain in dB
ShvType = "III" #I, II, III
Qz = np.sqrt(2)
Qp = Qz #dito
[B,A,b,a] = get_coeff_HShv2nd(fc,G,Qz,Qp,fs,ShvType)
get_Bode_plot(B,A,b,a,fs)

print("B=",B)
print("A=",A)
print("b=",b)
print("a=",a)

## Exercise

The **type III** filters consistently define the cut frequency as the point of half gain in dB.

Check the differences of the types I,II,III for the PEQ and shelving filters w.r.t. their cut frequencies.

Hint: 

* Where are the '3dB cut frequencies' located for type I,II for large positive or large negative gains in dB. 

* Can you define the '3 dB cut frequencies' for small gains $|G| <3$ dB?

* Can you define the half gain $G/2$ cut frequencies of type III for small gains in dB?

* Check how the filters of your favorite audio software are implemented.

In [ ]:
# HighShelve Example

Qz = 1/np.sqrt(2)
Qp = Qz # dito
fc = 1000  # cut frequency in Hz
G = 15  # gain in dB

# fc at 3 dB
ShvType = "II"
[B,A,b,a] = get_coeff_HShv2nd(fc,G,Qz,Qp,fs,ShvType)
get_Bode_plot(B,A,b,a,fs)

# fc at 7.5 dB
ShvType = "III"
[B,A,b,a] = get_coeff_HShv2nd(fc,G,Qz,Qp,fs,ShvType)
get_Bode_plot(B,A,b,a,fs)

# fc at 12 dB (15 dB - 3 dB)
ShvType = "I"
[B,A,b,a] = get_coeff_HShv2nd(fc,G,Qz,Qp,fs,ShvType)
get_Bode_plot(B,A,b,a,fs)

In [ ]:
# PEQ Example

BW = 2  # bandwidth in octaves
fm = 2000  # mid frequency in Hz, with BW=2 fc is at 1 kHz then
G = 15  # gain in dB
Q = get_Q_from_BW(BW)
QWarpType = "cos" # sin, cos, tan

PEQType = "II"
[B,A,b,a] = get_coeff_PEQ2nd(fm,G,Q,fs,PEQType,QWarpType)
get_Bode_plot(B,A,b,a,fs)

PEQType = "III"
[B,A,b,a] = get_coeff_PEQ2nd(fm,G,Q,fs,PEQType,QWarpType)
get_Bode_plot(B,A,b,a,fs)

PEQType = "I" #this type is not symmetrical w.r.t. same positive and negative gains!
[B,A,b,a] = get_coeff_PEQ2nd(fm,G,Q,fs,PEQType,QWarpType)
get_Bode_plot(B,A,b,a,fs)

## Excursus: Higher Order Filters

In audio signal processing also IIR filters of higher order are applied. However, they will be usually split into second order sections for series connection. Note that the group delay of an IIR filter increases with higher filter order. 

A prominent example of using higher order IIR filters is found in loudspeaker design. For a multi-way loudspeaker each driver must be driven within an appropriate intended frequency range. This is usually realized with a bandpass filter that is built from a higher order lowpass and a higher order highpass filter. Thus, each loudspeaker exhibits its own bandpass filter. The acoustic summation of all driver's generated sound pressure then yields the desired full audio bandwidth.

Consider, the following two-driver application, in which the same bandpass characteristic is used for the low and high frequency driver resulting up in a 4th order so called Linkwitz-Riley frequency crossover.

In [ ]:
N = 2**14  # frequency resolution

# two equal 2nd order Butterworth filters in series
# this is known as 4th order Linkwitz-Riley crossover
bi1 = 1 
ai1 = np.sqrt(2)
bi2 = 1 
ai2 = np.sqrt(2)

# low frequency driver range
fLow_LP = 1000
fLow_HP = 20

# high frequency driver range
fHigh_LP = 20000
fHigh_HP = fLow_LP

# get transfer functions of low frequency driver's bandpass
[B,A,b,a] = get_coeff_HP2nd(fLow_HP,fs,bi1,ai1)
W, LowHP1 = sig.freqz(b, a, N)
s, LowHP1 = sig.freqs(B, A, fs*W)
[B,A,b,a] = get_coeff_HP2nd(fLow_HP,fs,bi2,ai2)
W, LowHP2 = sig.freqz(b, a, N)
s, LowHP2 = sig.freqs(B, A, fs*W)

[B,A,b,a] = get_coeff_LP2nd(fLow_LP,fs,bi1,ai1)
W, LowLP1 = sig.freqz(b, a, N)
s, LowLP1 = sig.freqs(B, A, fs*W)
[B,A,b,a] = get_coeff_LP2nd(fLow_LP,fs,bi2,ai2)
W, LowLP2 = sig.freqz(b, a, N)
s, LowLP2 = sig.freqs(B, A, fs*W)

# get transfer function of high frequency driver's bandpass
[B,A,b,a] = get_coeff_HP2nd(fHigh_HP,fs,bi1,ai1)
W, HighHP1 = sig.freqz(b, a, N)
s, HighHP1 = sig.freqs(B, A, fs*W)
[B,A,b,a] = get_coeff_HP2nd(fHigh_HP,fs,bi2,ai2)
W, HighHP2 = sig.freqz(b, a, N)
s, HighHP2 = sig.freqs(B, A, fs*W)

[B,A,b,a] = get_coeff_LP2nd(fHigh_LP,fs,bi1,ai1)
W, HighLP1 = sig.freqz(b, a, N)
s, HighLP1 = sig.freqs(B, A, fs*W)
[B,A,b,a] = get_coeff_LP2nd(fHigh_LP,fs,bi2,ai2)
W, HighLP2 = sig.freqz(b, a, N)
s, HighLP2 = sig.freqs(B, A, fs*W)

Low = LowHP1*LowHP2 * LowLP1*LowLP2  # series connection of four 2nd order filters
High = HighHP1*HighHP2 * HighLP1*HighLP2  # series connection of four 2nd order filters

AcousticSum = Low + High  # simulation of the acoustic summation on middle axis 

f = fs*W / (2.*np.pi)
plt.figure(figsize = (16,9))
plt.semilogx(f, 20*np.log10(np.abs(Low)),'r', label=r'electric bandpass for low frequency driver')
plt.semilogx(f, 20*np.log10(np.abs(High)),'g', label=r'electric bandpass for high frequency driver')
plt.semilogx(f, 20*np.log10(np.abs(AcousticSum)),'b', label=r'acoustic on-axis summation')
plt.title(r'4th order Linkwitz-Riley X-Over for a two-way loudspeaker')
plt.xlabel(r'f / Hz')
plt.ylabel(r'20 log10 |H| / dB')
plt.axis([10., 20000., -30., +3.])
plt.yticks(np.arange(-30.,3.,3.));
plt.grid()
plt.legend(loc=3);



## References

**Copyright**

This notebook is provided as [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebook for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Sascha Spors, Digital Signal Processing - Lecture notes featuring computational examples, 2016-2018*.